In [1]:
import numpy as np
import pandas as pd

from findlay2025a import core
from findlay2025a.constants import Experiments as Exps
from findlay2025a.constants import Files

In [2]:
exclusions = [
    ("CNPIX5-Alessandro", Exps.NOD, ("ext_wake_incline", "ext_incline")),
    (
        "CNPIX6-Eugene",
        Exps.NOD,
        ("rebound", "bsl_decline", "ext_wake_incline", "ext_incline"),
    ),
    ("CNPIX10-Charles", Exps.NOD, ("ext_wake_incline", "ext_incline")),
    ("CNPIX11-Adrian", Exps.NOD, ("ext_wake_incline", "ext_incline")),
    ("CNPIX11-Adrian", Exps.COW, ("ext_wake_incline", "ext_incline")),
    ("CNPIX17-Hans", Exps.COW, ("ext_wake_incline", "ext_incline")),
]

contrast_display_names = {
    # Rebound
    "cx_mean_zlog_delta_nrem_rebound": "Cortical.SWA.Rebound",
    "spw_rate_rel2total_nrem_rebound": "SPW.Rate.Rebound",
    "spw_mean_zlog_amp_nrem_rebound": "SPW.Amplitude.Rebound",
    "ripple_rate_rel2total_nrem_rebound": "Ripple.Rate.Rebound",
    "ripple_mean_zfreq_nrem_rebound": "Ripple.Frequency.Rebound",
    "dspk_mean_zlog_height_nrem_rebound": "DSPK.Amplitude.Rebound",
    # Recovery Decline
    "cx_mean_zlog_delta_nrem_rec_decline": "Cortical.SWA.REC.Decline",
    "spw_rate_rel2total_nrem_rec_decline": "SPW.Rate.REC.Decline",
    "spw_mean_zlog_amp_nrem_rec_decline": "SPW.Amplitude.REC.Decline",
    "ripple_rate_rel2total_nrem_rec_decline": "Ripple.Rate.REC.Decline",
    "ripple_mean_zfreq_nrem_rec_decline": "Ripple.Frequency.REC.Decline",
    "dspk_mean_zlog_height_nrem_rec_decline": "DSPK.Amplitude.REC.Decline",
    # Surge
    "cx_mean_zlog_delta_nrem_surge": "Cortical.SWA.Surge",
    "spw_rate_rel2total_nrem_surge": "SPW.Rate.Surge",
    "spw_mean_zlog_amp_nrem_surge": "SPW.Amplitude.Surge",
    "ripple_rate_rel2total_nrem_surge": "Ripple.Rate.Surge",
    "ripple_mean_zlog_duration_nrem_surge": "Ripple.Duration.Surge",
    "ripple_mean_zfreq_nrem_surge": "Ripple.Frequency.Surge",
    "dspk_mean_zlog_height_nrem_surge": "DSPK.Amplitude.Surge",
    "dspk_rate_rel2total_nrem_surge": "DSPK.Rate.Surge",
    # Baseline Decline
    "cx_mean_zlog_delta_nrem_bsl_decline": "Cortical.SWA.BSL.Decline",
    "spw_rate_rel2total_nrem_bsl_decline": "SPW.Rate.BSL.Decline",
    "spw_mean_zlog_amp_nrem_bsl_decline": "SPW.Amplitude.BSL.Decline",
    "ripple_rate_rel2total_nrem_bsl_decline": "Ripple.Rate.BSL.Decline",
    "ripple_mean_zfreq_nrem_bsl_decline": "Ripple.Frequency.BSL.Decline",
    "dspk_mean_zlog_height_nrem_bsl_decline": "DSPK.Amplitude.BSL.Decline",
    "dspk_rate_rel2total_nrem_bsl_decline": "DSPK.Rate.BSL.Decline",
}

In [3]:
def apply_exclusions(df: pd.DataFrame) -> pd.DataFrame:
    essential_cols = ["subject", "experiment"]
    assert all(col in df.columns for col in essential_cols), (
        f"Missing one or more essential columns: {essential_cols}"
    )

    for sub, exp, conds in exclusions:
        cols = [
            col for col in df.columns if any(cond in col for cond in conds)
        ]  # Columns whose values to drop
        rows = (df["subject"] == sub) & (df["experiment"] == exp)
        df.loc[rows, cols] = np.nan

    return df


def use_display_names(df: pd.DataFrame, minimize: bool = True) -> pd.DataFrame:
    essential_cols = ["subject", "experiment"]
    assert all(col in df.columns for col in essential_cols), (
        f"Missing one or more essential columns: {essential_cols}"
    )

    if minimize:
        df = df.loc[:, essential_cols + list(contrast_display_names)]

    df["experiment"] = df["experiment"].map(core.EXPERIMENT_DISPLAY_NAMES)
    df.rename(columns=contrast_display_names, inplace=True)

    return df


def get_essential_contrasts() -> pd.DataFrame:
    nb = core.get_project("seahorse")
    df = pd.read_parquet(
        nb.get_project_file(Files.COMBINED_CONDITION_CONTRASTS)
    ).reset_index()
    return use_display_names(apply_exclusions(df))

In [4]:
nb = core.get_project("seahorse")
df = get_essential_contrasts()
df.to_parquet(nb.get_project_file(Files.EXPORTED_CONDITION_CONTRASTS))